In [1]:
param_date = '20240621'

In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/4.part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_.drop_duplicates(['JOIN_PTPKPNCN'], keep='first', ignore_index=True)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'getbuyersguide',
                        'payload': f'''{{"partData":{{"listing_data_essential":{{"parttype":"{input_.loc[a, 'Part_PartType']}","partkey":"{input_.loc[a, 'PartKey']}"}},"listing_data_supplemental":{{"partnumber":"{input_.loc[a, 'PartNumber']}","catalogname":"{input_.loc[a, 'CatalogName']}"}}}}}}''',
                        'partsearch[manufacturer][partsearch_007]': input_.loc[a, 'CatalogName'],
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                # = = = = = = = = = = = = = = =

                json_ = json.loads(resp)

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(json_['buyersguidepieces']['body'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_tr = html.xpath('//tr')

                # = = = = = = = = = = = = = = =

                dict_vehicle = {}
                for i in range(len(list_tr)):
                    dict_vehicle[str(i)] = {}
                    list_td = [td.strip() for td in list_tr[i].xpath('./td/text()')]
                    for j in range(len(list_td)):
                        dict_vehicle[str(i)][str(j)] = list_td[j]

                json_vehicle = json.dumps(dict_vehicle)

                # = = = = = = = = = = = = = = =

                vehicle = ''
                for i in dict_vehicle:
                    for j in dict_vehicle[i]:
                        vehicle += dict_vehicle[i][j] + ' '
                    vehicle = vehicle.strip() + '\n'

                vehicle = vehicle.strip()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'JOIN_PTPKPNCN': input_.loc[a, 'JOIN_PTPKPNCN'],
                                         'Json_Vehicle': json_vehicle,
                                         'Vehicle': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'JOIN_PTPKPNCN': input_.loc[a, 'JOIN_PTPKPNCN'],
                                     'Part_PartType': input_.loc[a, 'Part_PartType'],
                                     'PartKey': input_.loc[a, 'PartKey'],
                                     'PartNumber': input_.loc[a, 'PartNumber'],
                                     'CatalogName': input_.loc[a, 'CatalogName'],
                                     'PartType': input_.loc[a, 'PartType'],
                                     'PartKey': input_.loc[a, 'PartKey'],
                                     'Tab_Url': input_.loc[a, 'Tab_Url'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'JOIN_PTPKPNCN']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['JOIN_PTPKPNCN'], ascending=[True], ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/5.vehicle-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['JOIN_PTPKPNCN'], ascending=[True], ignore_index=True)
    output_error.to_excel(f'./{param_date}/vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：24750

[状态：ok，尝试次数：  1] - 131;15108561;B2;ROCKAUTO
[剩余数量：24730] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 1002098;15104205;H1;ROCKAUTO
[剩余数量：24729] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 256;14922129;T5;ROCKAUTO
[剩余数量：24728] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 1002098;15104209;H2;ROCKAUTO
[剩余数量：24727] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 256;15138777;T10;ROCKAUTO
[剩余数量：24726] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 256;10512261;T2;ROCKAUTO
[剩余数量：24725] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 256;15138781;T11;ROCKAUTO
[剩余数量：24724] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 256;9097025;T1;ROCKAUTO
[剩余数量：24723] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 256;15138769;T8;ROCKAUTO
[剩余数量：24722] - [当前时间：17:32:07]

[状态：ok，尝试次数：  1] - 256;14922125;T4;ROCKAUTO
[剩余数量：24721] - [当前时间：17:32:08]

[状态：ok，尝试次数：  1] - 256;15521065;T3C;ROCKAUTO
[剩余数量：24720] - [当前时间：17:32:08]

[状态：ok，尝试次数：  1] - 131;10671829;BLACKHAT;ROCKAUTO
[剩余数量：24719] - [当前时间：17:32:08]

[状态：ok，尝试次数：  1] - 256;15521069;T3R;ROCKAUTO
[剩余数量：24718] - [